In [161]:
from google.cloud import bigquery
import pandas as pd
import json

from datetime import date,datetime,timedelta

In [162]:
table_id = "pongthorn.SMartML.new_incident"
predictResult_table_id="pongthorn.SMartML.new_result_prediction_incident"

mapping_file="incident_sevirity_to_class.json"

PATH_FOLDER_ARTIFACTS="model"
unUsedColtoPredict=['severity','id','severity_id','severity_name','imported_at']

In [163]:
# with open(mapping_file, 'r') as json_file:
#      map_sevirity_to_class= json.load(json_file)
# print(map_sevirity_to_class)

map_sevirity_to_class={'Cosmatic': 0, 'Minor': 1, 'Major': 2, 'Critical': 3}
print(map_sevirity_to_class)

{'Cosmatic': 0, 'Minor': 1, 'Major': 2, 'Critical': 3}


In [164]:
# Get today's date
prediction_datetime=datetime.now()

today = date.today()
# Yesterday date
yesterday = today - timedelta(days = 1)
str_today=today.strftime('%Y-%m-%d')
str_yesterday=yesterday.strftime('%Y-%m-%d')
print(f"Get data between {str_yesterday} to {str_today} to predict sevirity level")

Get data between 2023-03-27 to 2023-03-28 to predict sevirity level


In [165]:
def load_data_bq(sql:str):
 client_bq = bigquery.Client()
 query_result=client_bq.query(sql)
 df=query_result.to_dataframe()
 return df

In [166]:
sql=f"""
SELECT *  FROM `{table_id}` 
WHERE DATE(imported_at) >= '{str_yesterday}' and DATE(imported_at) < '{str_today}' 

"""
#LIMIT 2
dfNewData=load_data_bq(sql)
dfNewData=dfNewData.drop_duplicates(subset=['id'],keep='first')

dfNewData.insert(2, 'severity', dfNewData['severity_name'].map(map_sevirity_to_class),True)


print(dfNewData.info())
print(dfNewData)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 50
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         51 non-null     Int64         
 1   severity_id                51 non-null     Int64         
 2   severity                   51 non-null     int64         
 3   severity_name              51 non-null     object        
 4   sla                        51 non-null     object        
 5   product_type               51 non-null     object        
 6   brand                      51 non-null     object        
 7   service_type               51 non-null     object        
 8   incident_type              51 non-null     object        
 9   open_to_close_hour         51 non-null     float64       
 10  response_to_resolved_hour  51 non-null     float64       
 11  imported_at                51 non-null     datetime64[ns]
dtypes: Int64(2

In [167]:
model = tf.keras.models.load_model(PATH_FOLDER_ARTIFACTS)    
print(f"Load from {PATH_FOLDER_ARTIFACTS}")
print(model.summary())

Load from model
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sla (InputLayer)               [(None, 1)]          0           []                               
                                                                                                  
 product_type (InputLayer)      [(None, 1)]          0           []                               
                                                                                                  
 brand (InputLayer)             [(None, 1)]          0           []                               
                                                                                                  
 service_type (InputLayer)      [(None, 1)]          0           []                               
                                                                            

In [168]:
pdPrediction=pd.DataFrame(columns=['_id','predict_severity','prob_severity'])

for  row_dict in dfNewData.to_dict(orient="records"):
      incident_id=row_dict['id']
      print(f"{incident_id} - {row_dict['severity']}({row_dict['severity_name']})") 
      for key_removed in unUsedColtoPredict:
       row_dict.pop(key_removed)
      # print(row_dict)  

      input_dict = {name: tf.convert_to_tensor([value]) for name, value in row_dict.items()}


      predictionResult = model.predict(input_dict)
      result_str=','.join([ str(prob) for prob in predictionResult[0]])  
      print(result_str)   

      prob = tf.nn.softmax(predictionResult)
      prob_pct=(100 * prob)  
      _class = tf.argmax(predictionResult,-1).numpy()[0]
      
      dictPrediction={'_id':incident_id, 'predict_severity':_class,'prob_severity':result_str} 
      pdPrediction =pd.concat([pdPrediction,pd.DataFrame.from_dict([dictPrediction])] )

      print(f"{prob_pct} %   as {_class}")     
      print("======================================================================================")
            
dfPredictData=pd.merge(dfNewData,pdPrediction,how='inner',left_on='id',right_on='_id')
dfPredictData=dfPredictData.drop(columns=['_id'])
dfPredictData['predict_severity']=dfPredictData['predict_severity'].astype('int')
dfPredictData=dfPredictData[['id','prob_severity','predict_severity','severity']]
dfPredictData['prediction_item_date']= datetime.strptime(str(yesterday), '%Y-%m-%d')
dfPredictData['prediction_datetime']=prediction_datetime

2491 - 1(Minor)
1/1 [==============================] - 1s 527ms/step
0.20702027,0.700546,0.08181793,0.01061574
[[23.030441 37.72574  20.320179 18.923645]] %   as 1
2516 - 1(Minor)
1/1 [==============================] - 0s 42ms/step
0.12305665,0.11055092,0.7646284,0.0017640598
[[20.952015 20.691626 39.7976   18.558765]] %   as 2
2502 - 1(Minor)
1/1 [==============================] - 0s 58ms/step
0.9542967,0.043510735,0.0021846048,7.928289e-06
[[46.01469  18.507475 17.758223 17.71961 ]] %   as 0
2489 - 0(Cosmatic)
1/1 [==============================] - 0s 44ms/step
0.93008536,0.0678322,0.0018174005,0.00026515758
[[45.206486 19.086617 17.867308 17.839596]] %   as 0
2514 - 0(Cosmatic)
1/1 [==============================] - 0s 57ms/step
0.051125165,0.91271,0.035917714,0.00024713963
[[18.860056 44.640034 18.575413 17.924496]] %   as 1
2474 - 0(Cosmatic)
1/1 [==============================] - 0s 51ms/step
0.9718198,0.019148123,0.009026669,5.3670187e-06
[[46.599846 17.973986 17.79298  17.63318

In [169]:
print(dfPredictData.info())
print(dfPredictData)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 50
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    51 non-null     object        
 1   prob_severity         51 non-null     object        
 2   predict_severity      51 non-null     int32         
 3   severity              51 non-null     int64         
 4   prediction_item_date  51 non-null     datetime64[ns]
 5   prediction_datetime   51 non-null     datetime64[ns]
dtypes: datetime64[ns](2), int32(1), int64(1), object(2)
memory usage: 2.6+ KB
None
      id                                      prob_severity  predict_severity  \
0   2491          0.20702027,0.700546,0.08181793,0.01061574                 1   
1   2516       0.12305665,0.11055092,0.7646284,0.0017640598                 2   
2   2502    0.9542967,0.043510735,0.0021846048,7.928289e-06                 0   
3   2489    0.93008536,0.0

In [170]:
#https://cloud.google.com/bigquery/docs/samples/bigquery-create-table#bigquery_create_table-python

try:
    client = bigquery.Client()
    client.get_table(predictResult_table_id)  # Make an API request.
    print("Predict Result Table {} already exists.".format(predictResult_table_id))
except Exception as ex:
    schema = [
    bigquery.SchemaField("id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("prob_severity", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("predict_severity", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("severity", "INTEGER", mode="REQUIRED"),    
    bigquery.SchemaField("prediction_item_date", "DATETIME", mode="REQUIRED"),    
    bigquery.SchemaField("prediction_datetime", "DATETIME", mode="REQUIRED") 
    ]

    table = bigquery.Table(predictResult_table_id,schema=schema)
    table.time_partitioning = bigquery.TimePartitioning(
    type_=bigquery.TimePartitioningType.DAY,field="prediction_item_date")
    
    table = client.create_table(table)  # Make an API request.
    
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )

Predict Result Table pongthorn.SMartML.new_result_prediction_incident already exists.


In [171]:
def loadDataFrameToBQ():
    try:
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_APPEND",
        )

        job = client.load_table_from_dataframe(
            dfPredictData, predictResult_table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total Prediction ML ", len(dfPredictData), "Imported igquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        for e in job.errors:
            print('ERROR: {}'.format(e['message']))

try:
    loadDataFrameToBQ()
except Exception as ex:
    raise ex

Total Prediction ML  51 Imported igquery successfully
